# Brasil - Comparações entre cidades
> Gráficos e análises das cidades brasileiros

- toc: false
- branch: master
- badges: false
- comments: true
- author: Vinicius Bastos Gomes & Fernando Wittmann
- categories: [coronavirus]



Fonte (estados):https://covid.saude.gov.br/
(cidades) https://data.brasil.io/dataset/covid19/caso.csv.gz


In [0]:
#hide
# Imports
import warnings
warnings.filterwarnings('ignore')
import gzip
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta
sns.set(style="ticks", rc={"lines.linewidth": 1.6})
sns.set_palette('dark')

In [0]:
#hide
# Functions
def sigla_nome(x):
    return dic[x]

def populacao(x):
    return popu[x]

In [3]:
#hide

!pip install plotly --upgrade

     |████████████████████████████████| 11.5MB 2.6MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [4]:
#hide
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
#hide
# Abrir pasta com os arquivos
%cd "/content/drive/My Drive/CoronaVirus Brasil"

/content/drive/My Drive/CoronaVirus Brasil


In [0]:
#hide
#dictionaries

dic = {'AC': "Acre",  'AL': "Alagoas",  'AM': "Amazonas", 'AP': "Amapá", 'BA': "Bahia", 'CE': "Ceará", 'DF': "Distrito Federal",'ES': "Espírito Santo",'GO': "Goiás",'MA': "Maranhão",'MG': "Minas Gerais",'MS': "Mato Grosso do Sul",'MT': "Mato Grosso",'PA': "Pará",'PB': "Paraíba",'PE': "Pernambuco",'PI': "Piauí",'PR': "Paraná",'RJ': "Rio de Janeiro",'RN': "Rio Grande do Norte",'RO': "Rondônia",'RR': "Roraima",'RS': "Rio Grande do Sul",'SC': "Santa Catarina",'SE': "Sergipe",'SP': "São Paulo",'TO': "Tocantins"} 
popu ={'AC': 881935,'AL': 3337357,'AM': 4144597,'AP': 845731,'BA': 14873064,'CE': 9132078,'DF': 3015268,'ES': 4018650,'GO': 7018354,'MA': 7075181,'MG': 21168791,'MS': 2778986,'MT': 3484466,'PA': 8602865,'PB': 4018127,'PE': 9557071,'PI': 3273227,'PR': 11433957,'RJ': 17264943,'RN': 3506853,'RO': 1777225,'RR': 605761,'RS': 11377239,'SC': 7164788,'SE': 2298696,'SP': 45919049,'TO': 1572866}

In [0]:
#hide
#manipulating datetime
todei = date.today()
today = todei.strftime("%d-%m-%Y")
today_us = todei.strftime("%Y-%m-%d")
today_us
yesterdei = date.today() - timedelta(days=1)
yesterday = yesterdei.strftime('%Y-%m-%d')

In [8]:
#hide
!rm '/content/drive/My Drive/CoronaVirus Brasil/caso.csv.gz'

rm: cannot remove '/content/drive/My Drive/CoronaVirus Brasil/caso.csv.gz': No such file or directory


In [0]:
#hide
# Parameters
BOLETIM = 'https://data.brasil.io/dataset/covid19/boletim.csv.gz'
CASOS = 'https://data.brasil.io/dataset/covid19/caso.csv.gz'

In [10]:
#hide
# Execution
!wget {CASOS}

--2020-05-31 14:08:27--  https://data.brasil.io/dataset/covid19/caso.csv.gz
Resolving data.brasil.io (data.brasil.io)... 192.99.5.40
Connecting to data.brasil.io (data.brasil.io)|192.99.5.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1259711 (1.2M) [application/gzip]
Saving to: ‘caso.csv.gz’

caso.csv.gz         100%[===================>]   1.20M  1.24MB/s    in 1.0s    

2020-05-31 14:08:29 (1.24 MB/s) - ‘caso.csv.gz’ saved [1259711/1259711]



In [0]:
#hide
with gzip.open('caso.csv.gz', 'rb') as f_in:
    with open('caso.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [12]:
#hide
casos = pd.read_csv('caso.csv')
casos.tail(4)

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
136855,2020-04-21,AL,Viçosa,city,1,1,4,False,25733.0,2709400.0,3.88606,1.0
136856,2020-04-20,AL,Viçosa,city,1,1,3,False,25733.0,2709400.0,3.88606,1.0
136857,2020-04-19,AL,Viçosa,city,1,1,2,False,25733.0,2709400.0,3.88606,1.0
136858,2020-04-18,AL,Viçosa,city,1,0,1,False,25733.0,2709400.0,3.88606,0.0


In [13]:
#hide
casos_por_cidade = casos.groupby(['state','city','date']).sum()[['confirmed', 'deaths']]
casos_por_cidade = casos_por_cidade.reset_index(level=[0,1])
casos_por_cidade['data'] = casos_por_cidade.index

casos_por_cidade.head(5)


,state,city,confirmed,deaths,data
date,,,,,
2020-03-29,AC,Acrelândia,2,0,2020-03-29
2020-03-30,AC,Acrelândia,6,0,2020-03-30
2020-03-31,AC,Acrelândia,7,0,2020-03-31
2020-04-01,AC,Acrelândia,8,0,2020-04-01
2020-04-02,AC,Acrelândia,8,0,2020-04-02


In [0]:
#hide
cidade_last = casos_por_cidade[casos_por_cidade.index==str(yesterday)]


In [0]:
#hide
cap = ['Rio Branco','Maceió','Macapá','Manaus','Salvador','Fortaleza','Brasília','Vitória','Goiânia','São Luís','Cuiabá','Campo Grande','Belo Horizonte','Belém','João Pessoa','Curitiba','Recife','Teresina','Rio de Janeiro','Natal','Porto Alegre','Porto Velho','Boa Vista','Florianópolis','São Paulo','Aracaju','Palmas']

In [16]:
#hide
casos_por_cidade.columns

Index(['state', 'city', 'confirmed', 'deaths', 'data'], dtype='object')

In [0]:
#hide
capitais = casos_por_cidade[casos_por_cidade['city'].isin(['Rio Branco','Maceió','Macapá','Manaus','Salvador','Fortaleza','Brasília','Vitória','Goiânia','São Luís','Cuiabá','Campo Grande','Belo Horizonte','Belém','João Pessoa','Curitiba','Recife','Teresina','Rio de Janeiro','Natal','Porto Alegre','Porto Velho','Boa Vista','Florianópolis','São Paulo','Aracaju','Palmas'])]
capitais_15 = capitais[capitais.index>='2020-03-20']

In [18]:
#hide
capitais_15.sort_values('confirmed',ascending=False,inplace=True)
teste = capitais_15.groupby(capitais_15.index)
teste

In [19]:
#hide
pro_bra = casos.groupby('date').sum()[['deaths','confirmed']]
pro_bra.tail(4)

,deaths,confirmed
date,,
2020-05-27,50576,806996
2020-05-28,53412,874651
2020-05-29,54782,899629
2020-05-30,29755,490833


In [20]:
#hide
pro_bra['Alemanha'] = pro_bra['deaths']*100
pro_bra['Coreia'] = pro_bra['deaths']*166.6
pro_bra['Data'] = pro_bra.index
pro_bra.columns = ['Óbitos','Brasil registrado','Projeção Alemanha','Projeção Coréia do Sul','Data']
pro_bra = pro_bra[pro_bra['Data']>='2020-03-17']
pro_bra.tail(3)

,Óbitos,Brasil registrado,Projeção Alemanha,Projeção Coréia do Sul,Data
date,,,,,
2020-05-28,53412,874651,5341200,8898439.2,2020-05-28
2020-05-29,54782,899629,5478200,9126681.2,2020-05-29
2020-05-30,29755,490833,2975500,4957183.0,2020-05-30


<center> <h2> Capitais brasileiras </h2> </center>



Comparações entre as capitais de estados do Brasil

In [0]:
#hide

capitais['Data'] = capitais.index
capitais.sort_values(['state','Data'])
capitais.columns = ['Estado','Cidade','Casos','Óbitos','data','Data']


In [0]:
#hide
import plotly.express as px

In [23]:
#hide_input
fig = px.line(capitais[capitais['Data']>='2020-03-15'], x="Data", y="Casos", title='COVID-19 Capitais Brasileiras - Casos {}'.format(today),
              color='Cidade',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [24]:
#hide_input
fig = px.line(capitais[capitais['Data']>='2020-03-15'], x="Data", y="Óbitos", title='COVID-19 Capitais Brasileiras - Óbitos - {}'.format(today),
              color='Cidade',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [25]:
#hide
casos_por_cidade.columns

Index(['state', 'city', 'confirmed', 'deaths', 'data'], dtype='object')

In [0]:
#hide
casos_por_cidade.sort_values(['city','data'],ascending=True,inplace=True)


<center> <h2> Linhas de todas as cidades </h2> </center>


A seguir as linhas de casos e óbitos de todas as cidades do Brasil das quais temos acesso aos dados. Para melhor utilização, encontre a cidade e clique duas vezes sobre sua legenda, isolando-a. Caso queira compará-la com outra cidade, adicione a linha da segunda cidade desejada clicando uma vez sobre sua legenda.


In [27]:
#hide_input
fig = px.line(casos_por_cidade, x='data', y="confirmed", title='COVID-19 Cidades Brasileiras - Casos - {}'.format(today),
              color='city',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [28]:
#hide_input
fig = px.line(casos_por_cidade, x='data', y="deaths", title='COVID-19 Cidades Brasileiras - Óbitos - {}'.format(today),
              color='city',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})

fig.show()

<center> <h2> 200 cidades com maiores números de casos </h2> </center>


Abaixo uma tabela com a lista das 200 cidades com mais casos no país.

In [29]:
#hide_input
pd.set_option('display.max_rows', None) 
x= cidade_last.sort_values('confirmed',axis=0,ascending=False)
x.columns = ['Estado','Cidade','Casos','Mortes','data']
x = x.drop('data',axis=1)
x.reset_index(level=[0],drop=True)
x.head(200)

,Estado,Cidade,Casos,Mortes
date,,,,
2020-05-30,SP,São Paulo,58619,4239
2020-05-30,CE,Fortaleza,23378,1939
2020-05-30,PE,Recife,14832,1040
2020-05-30,AL,Maceió,5107,262
2020-05-30,AP,Macapá,4992,127
2020-05-30,PB,João Pessoa,3902,126
2020-05-30,SP,Santos,3330,145
2020-05-30,PE,Jaboatão dos Guararapes,2993,300
2020-05-30,SP,Guarulhos,2723,280
